In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\gg787\ADP_데싸라면\adp_데이터모음\data\kc_house_data.csv")

In [13]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,standard,0,3,7,1180,0,1955,0,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,standard,0,3,7,2170,400,1951,1991,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,standard,0,3,6,770,0,1933,0,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,standard,0,5,7,1050,910,1965,0,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,standard,0,3,8,1680,0,1987,0,1800,7503


In [14]:
df.drop(["date","id"], axis = 1, inplace = True)

In [15]:
df = pd.get_dummies(data = df, columns = ["waterfront"])

In [16]:
feature = df.drop("price", axis = 1)
label = df["price"]

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size = 0.3, random_state = 2021)

In [19]:
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import roc_auc_score

bag = BaggingRegressor()
bag.fit(X_train, y_train)
pred = bag.predict(X_test)

print("학습 데이터 성능 : {}".format(bag.score(X_train, y_train)))
print("테스트 데이터 성능 : {}".format(bag.score(X_test, y_test)))



학습 데이터 성능 : 0.9516295400002776
테스트 데이터 성능 : 0.730057879276657


Out of Bag 샘플을 활용한 성능 측정

In [ ]:
reg_oob = BaggingRegressor(n_estimators = 100, oob_score = True)
# oob_score = True 로 설정시 Train, Test 데이터로 split 해주지 않아도 됨

reg_oob.fit(feature, label)
print(f"oob_score :.{reg_oob.oob_score_}")

In [ ]:
reg_oob.estimators_[0].feature_importances_

reg_oob.estimators_[0].feature_importances_.shape

### 변수 중요도 확인

In [ ]:
import numpy as np
importances = pd.DataFrame(np.mean([tree.feature_importances_ for tree in reg_oob.estimators_], axis=0))
feature_importances = pd.concat([pd.DataFrame(feature.columns), importances], axis=1)
feature_importances.columns=["col_name","feature_importance"]
feature_importances

# 변수의 상대적 중요도를 보았을 때 회귀 계수처럼 파악을 할 수 있다. 
# 변수 중요도가 큰 것을 파악할 수 있음

# 다른 코드
# tmp = []
# for tree in reg_oob.estimators_:
#     tree_im = tree.feature_importances_
#     tmp.append(tree_im)
    
    
# np.mean(tmp, axis = 0)

In [ ]:
sort_feature = feature_importances.sort_values(by = "feature_importance", ascending = False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (15,10))
sns.barplot(x = "col_name",y = "feature_importance", data = sort_feature)